# Correcting misclassified sequences

### Following information are needed to correct misclassified sequences
* cluster information
* sequence information 
* dictionary of {sequence:cluster}: A give sequence belongs to what cluster?
* python script to recommend taxa assignment for the misclassified sequences
* **full dataset is in the [Google Drive](https://drive.google.com/drive/u/2/folders/1lF7boVEF2hf9CSl0quW0T4IkRc54Fjd7)**
* python script to recommend taxa:
  - **you need to download the full dataset (3 arguments) from the google drive and run it**
    ```
      python seq_clstr_conflict.py 95-part-r-00000seq_cluster mis-ann/part-r-00000_converted mis-ann/part-r-00000_converted_1col_sorted_filtered_cut1
    ```

### Example of correcting misclassificatons

|sequence ID| cluster ID| current taxa ID| proposed taxa ID|
|---|---|---|---| 
|1CE6| 30034893 | {'11624', '9606'} |{'10090', '11627', '32630'}|
|1CKA |402311|{'9606', '32630'} |{'419612', '10090', '9837'}|
|1DD8| 24493348|{'562', '5541'} |{'28901', '149539', '90370'}|
|1E6J |947952|{'11706', '10090'}| {'11678', '57667', '11676'}|
|1EFG |82|{'146786', '197221', '32630'} |{'9823', '9606', '9913'}|
|1FE1| 82|{'32046', '1076', '10724'}| {'9823', '9606', '9913'}|
|1FU6 |4600043|{'10116', '32630'} |{'9598', '9606', '9913'}|
|1HXL |88270194|{'1895', '83833', '32630'}| {'9598', '9601', '9606'}|
|1JEW |290581|{'12072', '9606'}| {'41846', '12067', '138949'}|




In [ ]:
import os
import sys
import operator

seq_cluster_Dict = {}
cluster_Dict= {}

def read_seq_cluster(seq_cluster_file):
    with open(seq_cluster_file, "r") as f:
        for line in f:
            seq_id = line[:line.index(":")]   # WP_094632576:10000009
            clstr_id = line[line.index(":") + 1:].rstrip()
            seq_cluster_Dict[seq_id] = clstr_id

def read_clusters(clstr_file):
    cluster_Dict.clear()
    with open(clstr_file, "r") as cluster_file:
        for line in cluster_file:
            clstr_id = line[:line.index(":")]  # 1000003:1359185=1;784=2
            tax_row = line[line.index(":") + 1:].rstrip()
            cluster_Dict[clstr_id] = tax_row

# this returns the top(3) most frequent taxa
def get_top3_tax(line):
    taxDic = dict()
    tax_list = line.split(";")
    taxDic.clear()
    for i in range(len(tax_list)):
        tax_id = tax_list[i][:tax_list[i].index("=")]
        tax_count = int(tax_list[i][tax_list[i].index("=") + 1:])
        taxDic[tax_id] = tax_count

    #Sort the dic and return the top(3)
    sorted_taxDic = sorted(taxDic.items(), key=operator.itemgetter(1), reverse=True)
    return (sorted_taxDic[:3])

    return (taxDic)

def compare_taxa_set(seq_top3, clstr_top3):
    # [('300852', 4), ('10090', 3), ('146786', 2)]
    seq_set=set()
    clstr_set = set()
    for item in seq_top3:
        seq_set.add(item[0])

    for item in clstr_top3:
        clstr_set.add(item[0])
    print(seq_set, clstr_set)
    common_taxa = len (seq_set.intersection(clstr_set))
    return (common_taxa)



def verify_conflicts(sequence_file):
    taxDic = dict()
    with open(sequence_file, "r") as seq_file:
        for line in seq_file:    # 1A43:11676=1;11698=4
            if line.find(":") != -1:
                row_id = line[:line.index(":")]
                row_tax = line[line.index(":") + 1:].rstrip()
                seq_top3= get_top3_tax(row_tax) #

                #get the cluster of this sequence
                if row_id in seq_cluster_Dict:
                    clstr_id = seq_cluster_Dict[row_id]
                    if clstr_id in cluster_Dict:
                        # TODO: important check if all clusters are here
                        row_tax = cluster_Dict[clstr_id]

                        #get the top3 of the cluster
                        clstr_top3 = get_top3_tax(row_tax)
                        
                    #check top3 of cluster vs sequence
                    #TODO: check top1, top2, top3 if they are the same or not?
                    try:
                        print(row_id, clstr_id)
                        print(seq_top3, clstr_top3)
                        print ("common" + str(compare_taxa_set(seq_top3,clstr_top3)))
                    except:
                        print("error compare 2 sets ")

                     


                else:
                    print("seq id is not in a dictionary file")
read_seq_cluster(sys.argv[1])
read_clusters(sys.argv[2])
verify_conflicts(sys.argv[3])
